In [1]:
from influxdb import InfluxDBClient
import time, json
import pandas as pd
import numpy as np

In [2]:
config = {'influxDB': {'host': '192.168.50.110',
  'port': 18086,
  'user': 'admin',
  'password': '234*(sdlj12',
  'dbname': 'source_metric',
  'table': 'NzhEMjlBOTk3MzkxRjk5MjQyMzMzQzI4'}}

In [3]:
client = InfluxDBClient(
    host=config["influxDB"]["host"],  # new
    port=config["influxDB"]["port"],
    username=config["influxDB"]["user"],  # new
    password=config["influxDB"]["password"],
    database=config["influxDB"]["dbname"],
)

In [4]:
def getData(metric, timestamp):
    t0 = str(timestamp - 3600*24*9)+'000000000'
    t1 = str(timestamp)+'000000000'
    table = 'NzhEMjlBOTk3MzkxRjk5MjQyMzMzQzI4'
    sql = "select host,metricsVal,time from {} where metric='{}' and time>{} and time<={} order by time asc".format(table,metric, t0, t1)
    return pd.DataFrame(client.query(sql, epoch="s")[table])

In [5]:
def getThreshold(data):
    history = data[(data['time'] > 1666108800-3600*7*24)&(data['time'] < 1666108800)]['metricsVal'].values
    history.sort()
    history_p99 = np.percentile(history, 99)
    history_p75 = np.percentile(history, 75)
    history_p25 = np.percentile(history, 25)
    history_p1 = np.percentile(history, 1)
    threshold_high = history_p99+(history_p75-history_p25)
    threshold_low = history_p1-(history_p75-history_p25)
    return threshold_low,threshold_high

In [6]:
timestamp = int(time.time())

In [10]:
metric = 'system.mem.pagefile.used'
data = getData(metric, timestamp)
threshold_low,threshold_high = getThreshold(data[data['host'] == list(data['host'].unique())[0]])
threshold_high

2996.94453125

In [21]:
list(data['host'].unique())[]

['WIN-28L3MIEC5GE', 'DESKTOP-1DSLVAP', 'DESKTOP-LHIPQIC', 'WIN-963C9APBE1F']

In [11]:
metric = 'mysql.can_connect'
data = getData(metric, timestamp)
threshold_low,threshold_high = getThreshold(data)
threshold_low

-2.0